In [4]:
import pandas as pd

data = pd.read_csv('data/news_data.csv')
print(data.head())

                                     text   category
0               Galatasaray şampiyon oldu       spor
1      Türkiye'deki enflasyon oranı arttı    ekonomi
2  Hükümet yeni eğitim reformunu açıkladı   politika
3      Apple yeni iPhone modelini tanıttı  teknoloji
4  Futbol dünyasında dev transfer bombası       spor


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# Metinleri ve etiketleri ayıralım
X = data['text']
y = data['category']

# Veriyi eğitim ve test olarak bölelim
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# TF-IDF Vectorizer ile metinleri sayısal verilere dönüştürelim
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)  # max_features ile özellik sayısını sınırlayalım
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Farklı modelleri deneyelim

# 1. Naive Bayes
model_nb = MultinomialNB()
model_nb.fit(X_train_vec, y_train)
y_pred_nb = model_nb.predict(X_test_vec)
accuracy_nb = accuracy_score(y_test, y_pred_nb)

# 2. Logistic Regression
model_lr = LogisticRegression(max_iter=1000)
model_lr.fit(X_train_vec, y_train)
y_pred_lr = model_lr.predict(X_test_vec)
accuracy_lr = accuracy_score(y_test, y_pred_lr)

# 3. Grid Search ile Naive Bayes parametre optimizasyonu
param_grid = {'alpha': [0.1, 0.5, 1, 2, 5]}  # Naive Bayes için parametre ayarlaması
grid_search = GridSearchCV(MultinomialNB(), param_grid, cv=5)
grid_search.fit(X_train_vec, y_train)
best_nb_model = grid_search.best_estimator_
y_pred_best_nb = best_nb_model.predict(X_test_vec)
accuracy_best_nb = accuracy_score(y_test, y_pred_best_nb)

# Sonuçları yazdıralım
print(f"Naive Bayes Modelinin doğruluğu: {accuracy_nb * 100:.2f}%")
print(f"Logistic Regression Modelinin doğruluğu: {accuracy_lr * 100:.2f}%")
print(f"Optimizasyonlu Naive Bayes Modelinin doğruluğu: {accuracy_best_nb * 100:.2f}%")

# En iyi modeli seçelim
best_accuracy = max(accuracy_nb, accuracy_lr, accuracy_best_nb)
if best_accuracy == accuracy_nb:
    print("En iyi model: Naive Bayes")
elif best_accuracy == accuracy_lr:
    print("En iyi model: Logistic Regression")
else:
    print("En iyi model: Optimizasyonlu Naive Bayes")


Naive Bayes Modelinin doğruluğu: 77.78%
Logistic Regression Modelinin doğruluğu: 83.33%
Optimizasyonlu Naive Bayes Modelinin doğruluğu: 77.78%
En iyi model: Logistic Regression


In [6]:
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

# En iyi modeli seçelim
best_accuracy = max(accuracy_nb, accuracy_lr, accuracy_best_nb)

if best_accuracy == accuracy_nb:
    best_model = model_nb
    best_model_name = "Naive Bayes"
elif best_accuracy == accuracy_lr:
    best_model = model_lr
    best_model_name = "Logistic Regression"
else:
    best_model = best_nb_model
    best_model_name = "Optimizasyonlu Naive Bayes"

# En iyi modeli 'model' klasörüne kaydedin
joblib.dump(best_model, 'model/news_classifier_model.pkl')

# TF-IDF Vectorizer'ı 'model' klasörüne kaydedin
joblib.dump(vectorizer, 'model/tfidf_vectorizer.pkl')

# En iyi modeli yazdıralım
print(f"En iyi model: {best_model_name}")

print("Model ve TF-IDF Vectorizer başarıyla kaydedildi.")


En iyi model: Logistic Regression
Model ve TF-IDF Vectorizer başarıyla kaydedildi.
